In [4]:
from konlpy.tag import Kkma
from konlpy.utils import pprint

In [5]:
kkma = Kkma()

In [6]:
pprint(kkma.sentences(u'네, 안녕하세요. 반갑습니다.'))

['네, 안녕하세요.', '반갑습니다.']


-------

In [ ]:
seed_list = list ()
seed_list.append('https://news.naver.com/')

seen_list = list ()

while seed_list:
    seed = seed_list.pop(0)
    
    resp = download(seed, headers=headers)
    
    #Scrapping 영역
    #뉴스만 가져올 것이다
    if not re.search(r'text/html',resp.headers['Content-type'.lower()]):         #뉴스가 아니면 날려주기
        continue
        
    dom = BeautifulSoup(resp.text, 'lxml')
    
    main_area = dom.select_one('#main_content')                    #링크 수집    
    
    if not main_area:
        continue
        
    #뉴스기사에만 해당 되는 애들 가져오기. "구조" 활용
    news_list = main_area.select('li > a[href], li > div > a[href]') 
    
    # main body에 id 가 존재하였다
    
    article_area = dom.select_one('#articleBodyContents')
    
    if article_area:
        file_name = re.sub(r'\W','',seed.split('/')[-1])           #파일 명에 의미 없는 것 삭제 
        
        with open('TXT/'+file_name+'.txt','w',encoding='utf-8') as f:
            f.write(article_area.text)                                  #Scrapping은 여기서 끝난다.
        
    
    # — 크롤러 역할. 링크를 새로 찾고 추가해주는
    for link in news_list:        
        link = urljoin(seed, link.attrs['href'])
        if link not in seed_list and link not in seen_list:    
                seed_list.append(link)            
        
    
    seed_list.append(seed)

#TXT 폴더에 저장되었다.

-------

In [ ]:
P(A,B,C,D,E,F) = P(A|B,C,D,E,F)P(B|C,D,E,F)P(C|D,E,F)P(D|E,F)P(E|F)P(F)

Markov Asuumption (1st) = Bi-gram
                ~= P(A|B)P(B|C)P(C|D)P(D|E)P(E|F)P(F)
                ~= ㅠP(이전,사건)/P(이전)
        
계산하기 편하게 이렇게 assumption 을 만드는 것이다!

결합확률 = 조건부확률 개별확률(결합확률)
P(A,B)=P(A|B)P(B)
P(한|대)=count('대한')/count('대')
P(민|한)=count('한민')/count('한')



In [18]:
from os import listdir
import re

def cleaning(data):
    data = re.sub(r'\b[a-z_][a-zA-Z_()]{4,}\b', '', data)
    # 1. 사이드 스페이스가 있을 때 반드시 소문자+_  1글자로 시작, 영문(대소문자 포함) 4글자 이상 삭제 => 제일 뒷줄의 javascript 제거
    data = re.sub(r'\b[(){}/]{2,}', '', data)
    # 2. 함수 기호 삭제 => () {} 제거
    data = re.sub(r'([.,?!])(\w)', '\g<1> \g<2>', data)
    # 3. 구두점 다음에 글자가 있는 경우 공백 추가 => .단어 => . 단어로 변경
    data = re.sub(r'\s{2,}', '\n', data).strip()
    # 4. 공백 2개 이상 반복되는 경우 한 개만 => \n\n\n\n\n 또는      =>\n 또는 
    data = re.sub(r'[^\s\w\d.,?!:\'\"]', '', data)
    # 5. 공백, 문자, 숫자, 구두점을 제외한 나머지 글자 삭제 => 모든 특수문자 제거
    data ='\n'.join(data.splitlines()[1:])
    # * 네이버에서 수집한 뉴스는 첫 줄에 주석이 달려있음 => 첫 줄 주석 삭제
    return data

corpus = list()
for file in listdir('./TXT'):
    with open ('./TXT/'+file, 'rt', encoding='UTF8') as f:
        corpus.append(cleaning(f.read()))
        
data = ' '.join(corpus)

In [19]:
len(set (data. split())), len (data. split()), len (data), len(listdir('./TXT'))

(4473, 6990, 30153, 48)

In [ ]:
Zipf's law => TF - IDF
한글형태소분석기 = KoNLPy

In [ ]:
Ngram 만드는 코드

In [54]:
from collections import defaultdict

def ngram_umjeol (s, N=2) :
    rst = list ()
    for i in range (len(s)-1):
        rst. append (s[i: i+N])
    return rst


unigram = defaultdict(int)
for t in data.split ():
    for token in ngram_umjeol(t,1):
        unigram[token] += 1

bigram = defaultdict(int)
for t in data.split ():
    for token in ngram_umjeol(t):
        bigram[token] += 1
        
trigram = defaultdict(int)
for t in data.split ():
    for token in ngram_umjeol(t,3):
        trigram[token] += 1

In [51]:
sum(bigram.values()), len(bigram), sum(unigram.values()), len(unigram),\
sum(trigram.values()), len(trigram)

(16117, 5576, 16117, 757, 16117, 8215)

P(A,B) = freq(Bigram)/freq(Unigram) => 1st markov assumption
우린 이렇게 쓸거다!!
우린 유니그램 프리퀀시와 바이그램 프리퀀시 둘다 필요하다
근데 앞에서 만들어놨다~

In [57]:
start = '대'

for _ in range(10): #10개의 음절을 생성(예측 by bigram language model)
    freq1 = bigram[start]
    maxfreq = 0.0
    
    for k in [_ for _ in bigram.keys() if _.startswith(start)]:
        freq2 = trigram[k]
        prob = freq2/freq1
        #print(k, freq2/freq1)
        if maxfreq<prob:
            print(start, k, prob)
            maxfreq = prob
            start = k[-2:]
    print()

ZeroDivisionError: division by zero

----------

In [ ]:
점심먹고왔따!

-----

In [ ]:
한국말은 굴절어의 특징을 갖는다
어미의 활용이 다양하다!
어근, 어간, 형태소 + 어미
밥 을/도/()/ +먹(다)+었+어 등등..

영어같은경우는 위치가 결정되어있지만 한국어는 접사에 대한 정보가 더 중요하다
접사에 따라 품사가 결정되고, 문장 성분내에 어떤 역할을 하는지도 결정된다
거기다 띄어쓰기도 있고..

이런 형태때문에 형태소를 띄어내야 하고, 원형을 복원해야하기도 하고..
stemming, lemmatization
-> 토큰화를 하는거다!

자연어를 가지고 feature 형태로 삼으려면 적절한 크기와 형태의 피쳐들이 필요한데,
이 방법을 토크나이징이라고 부른다 - stemming, lemmatization, ngram byte pair encoding 이 여기 속한다

토크나이징 - 인덱싱 - vocabulary(UNK)

LM = N-gram =>tokenizing(음절, 어절)

In [63]:
from konlpy.tag import Kkma, Komoran, Hannanum, Okt, Mecab

In [64]:
okt = Okt()
okt.tagset

{'Adjective': '형용사',
 'Adverb': '부사',
 'Alpha': '알파벳',
 'Conjunction': '접속사',
 'Determiner': '관형사',
 'Eomi': '어미',
 'Exclamation': '감탄사',
 'Foreign': '외국어, 한자 및 기타기호',
 'Hashtag': '트위터 해쉬태그',
 'Josa': '조사',
 'KoreanParticle': '(ex: ㅋㅋ)',
 'Noun': '명사',
 'Number': '숫자',
 'PreEomi': '선어말어미',
 'Punctuation': '구두점',
 'ScreenName': '트위터 아이디',
 'Suffix': '접미사',
 'Unknown': '미등록어',
 'Verb': '동사'}

In [65]:
kkma = Kkma()
kkma.tagset

{'EC': '연결 어미',
 'ECD': '의존적 연결 어미',
 'ECE': '대등 연결 어미',
 'ECS': '보조적 연결 어미',
 'EF': '종결 어미',
 'EFA': '청유형 종결 어미',
 'EFI': '감탄형 종결 어미',
 'EFN': '평서형 종결 어미',
 'EFO': '명령형 종결 어미',
 'EFQ': '의문형 종결 어미',
 'EFR': '존칭형 종결 어미',
 'EP': '선어말 어미',
 'EPH': '존칭 선어말 어미',
 'EPP': '공손 선어말 어미',
 'EPT': '시제 선어말 어미',
 'ET': '전성 어미',
 'ETD': '관형형 전성 어미',
 'ETN': '명사형 전성 어미',
 'IC': '감탄사',
 'JC': '접속 조사',
 'JK': '조사',
 'JKC': '보격 조사',
 'JKG': '관형격 조사',
 'JKI': '호격 조사',
 'JKM': '부사격 조사',
 'JKO': '목적격 조사',
 'JKQ': '인용격 조사',
 'JKS': '주격 조사',
 'JX': '보조사',
 'MA': '부사',
 'MAC': '접속 부사',
 'MAG': '일반 부사',
 'MD': '관형사',
 'MDN': '수 관형사',
 'MDT': '일반 관형사',
 'NN': '명사',
 'NNB': '일반 의존 명사',
 'NNG': '보통명사',
 'NNM': '단위 의존 명사',
 'NNP': '고유명사',
 'NP': '대명사',
 'NR': '수사',
 'OH': '한자',
 'OL': '외국어',
 'ON': '숫자',
 'SE': '줄임표',
 'SF': '마침표, 물음표, 느낌표',
 'SO': '붙임표(물결,숨김,빠짐)',
 'SP': '쉼표,가운뎃점,콜론,빗금',
 'SS': '따옴표,괄호표,줄표',
 'SW': '기타기호 (논리수학기호,화폐기호)',
 'UN': '명사추정범주',
 'VA': '형용사',
 'VC': '지정사',
 'VCN': "부정 지정사, 형용사 '아니다'",
 'VC

In [ ]:
꼬꼬마는 서울대에서 만든거, 더 자세하고 더 느리다!

In [67]:
han = Hannanum()
kom = Komoran()

In [68]:
s = '아버지가방에들어가신다.'
okt.pos(s)

([('아버지', 'Noun'),
  ('가방', 'Noun'),
  ('에', 'Josa'),
  ('들어가신다', 'Verb'),
  ('.', 'Punctuation')],
 [('아버지', 'NNG'),
  ('가방', 'NNG'),
  ('에', 'JKM'),
  ('들어가', 'VV'),
  ('시', 'EPH'),
  ('ㄴ다', 'EFN'),
  ('.', 'SF')],
 [('아버지가방에들어가', 'N'), ('이', 'J'), ('시ㄴ다', 'E'), ('.', 'S')],
 [('아버지', 'NNG'),
  ('가방', 'NNP'),
  ('에', 'JKB'),
  ('들어가', 'VV'),
  ('시', 'EP'),
  ('ㄴ다', 'EF'),
  ('.', 'SF')])

In [69]:
kkma.pos(s)

[('아버지', 'NNG'),
 ('가방', 'NNG'),
 ('에', 'JKM'),
 ('들어가', 'VV'),
 ('시', 'EPH'),
 ('ㄴ다', 'EFN'),
 ('.', 'SF')]

In [70]:
han.pos(s)

[('아버지가방에들어가', 'N'), ('이', 'J'), ('시ㄴ다', 'E'), ('.', 'S')]

In [71]:
kom.pos(s)

[('아버지', 'NNG'),
 ('가방', 'NNP'),
 ('에', 'JKB'),
 ('들어가', 'VV'),
 ('시', 'EP'),
 ('ㄴ다', 'EF'),
 ('.', 'SF')]

In [ ]:
형태소분석기 중 꼬꼬마가 제일 좋고, 한나눔이 제일 별로고, 오케이티가 제일 무난하다!


In [83]:
from konlpy.corpus import kolaw

In [73]:
kolaw는 헌법, kobill은 의안이다! 코엔엘파이에서 제공하는 기존 말뭉치다!

In [95]:
data = kolaw.open(kolaw.fileids()[0]).read()

In [96]:
len(data), len(data.split()), len(set(data.split()))

(18884, 4178, 2029)

In [98]:
len(okt.morphs(data)), len(set(okt.morphs(data)))

(8796, 1364)

In [ ]:
셋으로 중복으로 사용된 어미를 제거했다!

In [102]:
josa = [_[0]for _ in okt.pos(data) if _[1]=='Josa']

In [104]:
len(josa), len(set(josa))

(1981, 47)

In [ ]:
이게 조사만 있는 경우다! 유니크한 조사가 47개고, 조사의 프리퀀시가 1981개라는 뜻이다!

In [106]:
okt.pos('\n'. join(data.splitlines()[:3]))

[('대한민국', 'Noun'),
 ('헌법', 'Noun'),
 ('\n\n', 'Foreign'),
 ('유구', 'Noun'),
 ('한', 'Josa'),
 ('역사', 'Noun'),
 ('와', 'Josa'),
 ('전통', 'Noun'),
 ('에', 'Josa'),
 ('빛나는', 'Verb'),
 ('우리', 'Noun'),
 ('대', 'Modifier'),
 ('한', 'Modifier'),
 ('국민', 'Noun'),
 ('은', 'Josa'),
 ('3', 'Number'),
 ('·', 'Punctuation'),
 ('1', 'Number'),
 ('운동', 'Noun'),
 ('으로', 'Josa'),
 ('건립', 'Noun'),
 ('된', 'Verb'),
 ('대한민국', 'Noun'),
 ('임시정부', 'Noun'),
 ('의', 'Josa'),
 ('법', 'Noun'),
 ('통과', 'Noun'),
 ('불의', 'Noun'),
 ('에', 'Josa'),
 ('항거', 'Noun'),
 ('한', 'Josa'),
 ('4', 'Number'),
 ('·', 'Punctuation'),
 ('19', 'Number'),
 ('민주', 'Noun'),
 ('이념', 'Noun'),
 ('을', 'Josa'),
 ('계승', 'Noun'),
 ('하고', 'Josa'),
 (',', 'Punctuation'),
 ('조국', 'Noun'),
 ('의', 'Josa'),
 ('민주', 'Noun'),
 ('개혁', 'Noun'),
 ('과', 'Josa'),
 ('평화', 'Noun'),
 ('적', 'Suffix'),
 ('통일', 'Noun'),
 ('의', 'Josa'),
 ('사명', 'Noun'),
 ('에', 'Josa'),
 ('입', 'Noun'),
 ('각하', 'Noun'),
 ('여', 'Josa'),
 ('정의', 'Noun'),
 ('·', 'Punctuation'),
 ('인도', 'Noun'),

In [ ]:
okt가 분석한 결과는 말이 되지만 좀 애매하다..

------------

In [ ]:
이번엔 영어를 할거다!

In [108]:
import nltk

In [109]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [ ]:
담화 -> 문단 -> 문장 -> 어절(구문) -> 품사 -> 형태소
                                        토큰화

In [111]:
from nltk.tokenize import sent_tokenize, word_tokenize

In [113]:
len(data.splitlines()), len(sent_tokenize(data))

(356, 357)